In [ ]:
from openmmtools.constants import kB
from simtk import unit
import numpy as np
from tqdm import tqdm
from IPython.core.display import display, HTML
from IPython.display import SVG
from rdkit.Chem.Draw import IPythonConsole
from rdkit import Chem
from rdkit import Chem
import copy
import random, math
import os
from neutromeratio.utils import * 
import pickle
import mdtraj as md

# temperature, mass, and related constants
temperature = 300 * unit.kelvin
kT = kB * temperature
mass = (12.0 * unit.dalton)
sigma_v = np.sqrt(kB * temperature / mass)

# openmm units
mass_unit = unit.dalton
distance_unit = unit.nanometer
time_unit = unit.femtosecond
energy_unit = unit.kilojoule_per_mole
speed_unit = distance_unit / time_unit
force_unit = unit.kilojoule_per_mole / unit.nanometer

# ANI-1 units and conversion factors
ani_distance_unit = unit.angstrom
hartree_to_kJ_mol = 2625.499638
ani_energy_unit = hartree_to_kJ_mol * unit.kilojoule_per_mole # simtk.unit doesn't have hartree?
nm_to_angstroms = (1.0 * distance_unit) / (1.0 * ani_distance_unit)
angstroms_to_nm = (0.1 * ani_distance_unit) / (1.0 * distance_unit)       

In [ ]:
# use torchani to sample methane
# TODO: modify this to be an instance method of a Molecule class instead of a function sitting in main...
import torchani
import torch
device = torch.device('cpu')
model = torchani.models.ANI1ccx()
model = model.to(device)
torch.set_num_threads(4)

In [ ]:
exp_results = pickle.load(open('../data/exp_results.pickle', 'rb'))

# specify the system you want to simulate
name = 'molDWRow_37'
from_mol_tautomer_idx = 1
to_mol_tautomer_idx = 2
# specify a run number
nr_of_runs = 10
# number of time steps
nr_of_mc_trials = 10

t1_smiles = exp_results[name]['t1-smiles']
t2_smiles = exp_results[name]['t2-smiles']



In [ ]:
# generate both rdkit mol
mols = { 't1' : generate_rdkit_mol(t1_smiles), 't2' : generate_rdkit_mol(t2_smiles) }
write_pdb(mols['t1'], name, 't1')
write_pdb(mols['t2'], name, 't2')

from_mol = mols[f"t{from_mol_tautomer_idx}"]
to_mol = mols[f"t{to_mol_tautomer_idx}"]


In [ ]:
ani_input = from_mol_to_ani_input(from_mol)

# extract hydrogen donor idx and hydrogen idx for m2
donor = get_donor_atom_idx(from_mol, to_mol)

# extract hydrogen acceptor idx for to_mol
acceptor = 8
hydrogen_mover = MC_mover(donor_idx = donor['donor'], hydrogen_idx = donor['hydrogen_idx'], acceptor_idx = acceptor, atom_list = ani_input['atom_list'])

species = model.species_to_tensor(ani_input['atom_list']).to(device).unsqueeze(0)

# initial conditions: coordinates from example were given in units of angstroms   
x_in_angstroms = np.array(ani_input['coord_list']) * unit.angstrom
print(f"Hydrogen {hydrogen_mover.hydrogen_idx} is moved from atom-idx {hydrogen_mover.donor_idx} to atom-idx {hydrogen_mover.acceptor_idx}.")
# generating mdtraj traj object
topology = md.load(f"../data/md_sampling/{name}/{name}_t{from_mol_tautomer_idx}.pdb").topology

for run in range(2, nr_of_runs+1):
    print(f"Run Nr: {run}")
    trange = tqdm(range(nr_of_mc_trials))

    traj_in_nm = []
    work_value = []
    accept_reject = []
    states = []
    current_state = from_mol_tautomer_idx

    for ts in trange:

        x0 = x_in_angstroms
        trajectory, x, energies = langevin(device, model, ani_input['atom_list'], x0, ANI1ccx_force_and_energy, n_steps=100, stepsize=0.5*unit.femtosecond, temperature=temperature, platform='cuda', progress_bar=False)
        traj_in_nm += [x / unit.nanometer for x in trajectory]
        # for debug purpose: write out mol before and after MC move
        write_xyz_file(hydrogen_mover.atom_list, x, name=f"{name}_t{from_mol_tautomer_idx}", identifier=f"{run}_{ts}_0_pre_jump")
        # MC move
        accept, coordinates, log_p = hydrogen_mover.perform_mc_move(x, ts, model, species, device)
        work_value.append(log_p)
        # for debug purpose: write out mol before and after MC move
        write_xyz_file(hydrogen_mover.atom_list, coordinates, name=f"{name}_t{from_mol_tautomer_idx}", identifier=f"{run}_{ts}_1_post_jump")


        if accept:
            print('######################')
            print('MC move accepted. Hurray!')
            print('######################')
            #x_in_angstroms = coordinates
            accept_reject.append(1)
            if current_state == from_mol_tautomer_idx:               
                current_state = to_mol_tautomer_idx
            elif current_state == to_mol_tautomer_idx:
                current_state = from_mol_tautomer_idx
            else:
                raise RuntimeError('Something is wrong. Just two states possible.')
        else:
            accept_reject.append(0)
            x_in_angstroms = x

        states.append(current_state)

    # print MC statistics
    print('Nr of accepted and rejected MC moves: {}/{}'.format(hydrogen_mover.accept_counter, hydrogen_mover.reject_counter))
    ani_traj = md.Trajectory(traj_in_nm, topology)
    ani_traj = ani_traj.superpose(ani_traj[0]) # RMSD align onto first frame
    ani_traj.save(f"../data/md_mc_sampling/{name}/{name}_t{from_mol_tautomer_idx}_run{run}_anicxx.dcd")

    f = open(f"../data/md_mc_sampling/{name}/{name}_from_t{from_mol_tautomer_idx}_to_t{to_mol_tautomer_idx}_run{run}_work.csv", 'w+')
    for i, j, k, l in zip(range(nr_of_mc_trials), work_value, accept_reject, states):
        f.write('{}, {}, {}, {}\n'.format(i, j, k, l))
    f.close()
        
